In [ ]:
import pyrosetta
pyrosetta.init()

from multiprocessing import Pool
import logging

import itertools
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn


In [ ]:
aa_dict = {
    'A': 1,'C': 2,'D': 3,'E': 4,
    'F': 5,'G': 6,'H': 7,'I': 8,
    'K': 9,'L': 10,'M': 11,'N': 12,
    'P': 13,'Q': 14,'R': 15,'S': 16,
    'T': 17,'V': 18,'W': 19,'Y': 20}

In [ ]:
# import pose from pdb

pdb = '1stn-relaxed.pdb'
relaxed_pose = pyrosetta.pose_from_pdb(pdb)

# pmm = pyrosetta.rosetta.protocols.moves.PyMOLMover()
# pmm.apply(relaxed_pose)

In [ ]:
# make backrub ensemble from WT

n_backrub = 50

backrub_protocol = pyrosetta.rosetta.protocols.backrub.BackrubProtocol()

pyrosetta.rosetta.core.pose.setPoseExtraScore(relaxed_pose, "structure_num", 0)
backrub_pose_list = [relaxed_pose]

for i in range(1, n_backrub):
    pose = relaxed_pose.clone()
    pyrosetta.rosetta.core.pose.setPoseExtraScore(pose, "structure_num", i)
    backrub_protocol.apply(pose)
    backrub_pose_list.append(pose)

In [ ]:
# read in mutations from Kellogg dataset

df = pd.read_csv('./kellogg.csv', comment = '#')
df['chain'], df['wt_res'], df['pdb_res_index'], df['mut_res'] = df['Mutations'].str.split(' ', 3).str
df.pdb_res_index = df.pdb_res_index.astype(int)
df = df.drop('Mutations', axis=1)
df.head()

In [ ]:
# pick mutations based on pdb

pdb_id = pdb[:4].upper()
pdb_ddgs = df[df.PDBFileID == pdb_id]
pdb_ddgs['res_index'] = pdb_ddgs.apply(lambda x: relaxed_pose.pdb_info().pdb2pose(x.chain, int(x.pdb_res_index)), axis=1)

ddgs = pdb_ddgs[['chain', 'wt_res', 'pdb_res_index', 'mut_res', 'res_index', 'DDG']]

print('PDB: {}'.format(pdb_id))
print('Max ddG = {}'.format(ddgs.DDG.max()))
print('Min ddG = {}'.format(ddgs.DDG.min()))

ddgs.head()

In [ ]:
# prepare mutation arguments from dataframe
mutations = list(zip(pdb_ddgs.res_index, pdb_ddgs.mut_res))

# create a mutation argument for WT (no mutation)
mutations.append((relaxed_pose.total_residue(), None))

# setup tuples for multiprocessing
work = [(input_pose, res_index, new_aa)
        for input_pose, (res_index, new_aa)
        in itertools.product(backrub_pose_list, mutations)] 

In [ ]:
def mutate_minimize(input_pose, res_index, new_aa = None):
    '''Adapted from an example by Alex Ford and Brian Weitzner at Pre-RosettaCON 2018¶'''
    
    #work_pose = packed_pose.to_pose(input_pose)
    work_pose = input_pose
    
    #Mutate residue
    if new_aa != None:
        pyrosetta.toolbox.mutants.mutate_residue(work_pose, res_index, new_aa)
        pyrosetta.rosetta.core.pose.setPoseExtraScore(work_pose, "mutation_aa", aa_dict[new_aa])
        pyrosetta.rosetta.core.pose.setPoseExtraScore(work_pose, "mutation_index", res_index)
    else:
        pyrosetta.rosetta.core.pose.setPoseExtraScore(work_pose, "mutation_aa", -1)
        pyrosetta.rosetta.core.pose.setPoseExtraScore(work_pose, "mutation_index", res_index)
    
    #Minimize inner shell and whole protein
    minimizer(work_pose, res_index)
    
    return work_pose

In [ ]:
def minimizer(work_pose, res_index):
    
    #work_pose = pyrosetta.pose_from_pdb('/Users/mariajaime/Desktop/2018-2019/PyRosetta/ddG/pyrosetta_project/PyRosetta_Project/1stn-relaxed.pdb')
    #pmm = pyrosetta.PyMOLMover()
    #pmm.pymol_name('input')
    #pmm.apply(work_pose)

    # MovemapFactory
    mmf = pyrosetta.rosetta.core.select.movemap.MoveMapFactory()
    mmf.all_bb(setting=True)  # Set to true if needed
    mmf.all_bondangles(setting=True)
    mmf.all_bondlengths(setting=True)
    mmf.all_chi(setting=True)
    mmf.set_cartesian(setting=True)
    #help(pyrosetta.rosetta.core.select.movemap.MoveMapFactory)

    # Create residue selectors

    index_selector = pyrosetta.rosetta.core.select.residue_selector.ResidueIndexSelector(str(res_index))
    close_residue_selector = pyrosetta.rosetta.core.select.residue_selector.NeighborhoodResidueSelector(index_selector, 10.0, True)
    not_close_residue_selector = pyrosetta.rosetta.core.select.residue_selector.NotResidueSelector(close_residue_selector)

    # Create residue level task operations

    prevent_repacking_rlt = pyrosetta.rosetta.core.pack.task.operation.PreventRepackingRLT()

    # Create task operations

    not_close_prevent_repack_t_op = pyrosetta.rosetta.core.pack.task.operation.OperateOnResidueSubset()
    #help(not_close_prevent_repack_t_op)
    not_close_prevent_repack_t_op.op(prevent_repacking_rlt)
    not_close_prevent_repack_t_op.selector(not_close_residue_selector)

    tf = pyrosetta.rosetta.core.pack.task.TaskFactory()
    tf.push_back(pyrosetta.rosetta.core.pack.task.operation.RestrictToRepacking()) #keep from designing
    tf.push_back(pyrosetta.rosetta.core.pack.task.operation.NoRepackDisulfides())
    tf.push_back(not_close_prevent_repack_t_op)

    #Minimization

    sfxn = pyrosetta.create_score_function("ref2015_cart.wts")
    min_mover = pyrosetta.rosetta.protocols.minimization_packing.MinMover('lbfgs_armijo_nonmonotone')
    min_mover.min_type('lbfgs_armijo_nonmonotone')
    min_mover.tolerance(0.01)

    #help(min_mover)
    min_mover.movemap_factory(mmf) 
    min_mover.score_function(sfxn)
    #min_mover.apply(work_pose)

    # Packing

    packer = pyrosetta.rosetta.protocols.minimization_packing.PackRotamersMover()
    packer.task_factory(tf)
    packer.apply(work_pose)

    #pmm.pymol_name('output')
    #pmm.apply(work_pose)
    
    return work_pose



In [ ]:
# run mutate and minimize, using multiprocessing
with pyrosetta.distributed.utility.log.LoggingContext(logging.getLogger("rosetta"), level=logging.WARN):
    with Pool() as p:         
        logging.info("Now generating mutations and minimizing")
        all_poses = p.starmap(mutate_minimize, work)

In [ ]:
# make a score function
sfxn = pyrosetta.get_fa_scorefxn()

In [ ]:
df_scores = pd.DataFrame(data = np.zeros((len(all_poses),4)), columns=['structure_num', 'res_index', 'mut_res', 'score_REU'])

In [ ]:
for i, pose in enumerate(all_poses):
    structure_num = pyrosetta.rosetta.core.pose.getPoseExtraScore(pose, "structure_num")
    res_index = pyrosetta.rosetta.core.pose.getPoseExtraScore(pose, "mutation_index")
    mut = pyrosetta.rosetta.core.pose.getPoseExtraScore(pose, "mutation_aa")
    if mut == -1:
        mut_res = 'WT'
    else:
        mut_res = [key for key, value in aa_dict.items() if int(value) == mut][0]
    
    df_scores.loc[i,'structure_num'] = structure_num
    df_scores.loc[i,'res_index'] = res_index
    df_scores.loc[i,'mut_res'] = mut_res
    df_scores.loc[i,'score_REU'] = sfxn(pose)


In [ ]:
avg_score_df = df_scores.groupby(by = ['res_index','mut_res']).agg({'score_REU': 'mean'})

In [ ]:
avg_score_df

In [ ]:
final_df = pd.merge(ddgs, avg_score_df, how = 'left', on = ['res_index','mut_res'])

In [ ]:
plt.scatter(final_df.DDG, final_df.score_REU)
plt.xlabel('experimental ddG')
plt.ylabel('REU')